In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/user-click-rata-data/user_click_data.csv
/kaggle/input/user-click-rata-data/newsdata_user_data.csv


In [9]:
data = pd.read_csv('/kaggle/input/user-click-rata-data/newsdata_user_data.csv')

In [10]:
data.head()

,Unnamed: 0,article_id,title,text,link
0,0,0,COVID SCIENCE-Twin antibodies may help fight c...,Twin antibodies may be better than oneScientis...,https://www.reuters.com/article/idUSL1N2CV1RN
1,1,1,Breakingviews - Corona Capital: Ford narrows t...,NEW YORK/HONG KONG/MILAN/LONDON (Reuters Break...,https://www.reuters.com/article/idUSKCN24V3NO
2,2,2,"Breakingviews - Corona Capital: Hockey, HBO, M...",NEW YORK/ZURICH/LONDON/MILAN/HONG KONG (Reuter...,https://www.reuters.com/article/idUSKBN23334I
3,3,3,Sports-On this day... April 12,"(Reuters) - ON THIS DAY -- April 12April 12, 1...",https://www.reuters.com/article/idUSKCN21T08L
4,4,4,Breakingviews - Corona Capital: Trouble in the...,NEW YORK/ZURICH/MUMBAI/LONDON (Reuters Breakin...,https://www.reuters.com/article/idUSKCN22A39X


In [11]:
data.insert(0,'id',range(0,data.shape[0]))

ds = data[['id','title','text','link']]

In [12]:
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random

In [13]:
# Function for removing NonAscii characters
#def _removeNonAscii(s):
#    return "".join(i for i in s if  ord(i)<128)

# Function for converting into lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    texts = [w for w in text if w.isalpha()]
    texts = " ".join(texts)
    return texts

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

# Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Applying all the functions in description and storing as a cleaned_desc
#ds['cleaned_desc'] = ds['text'].apply(_removeNonAscii)
ds['cleaned_desc'] = ds['text'].apply(func = make_lower_case)
ds['cleaned_desc'] = ds.cleaned_desc.apply(func = remove_stop_words)
ds['cleaned_desc'] = ds.cleaned_desc.apply(func=remove_punctuation)
ds['cleaned_desc'] = ds.cleaned_desc.apply(func=remove_html)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [14]:
ds.head()

,id,title,text,link,cleaned_desc
0,0,COVID SCIENCE-Twin antibodies may help fight c...,Twin antibodies may be better than oneScientis...,https://www.reuters.com/article/idUSL1N2CV1RN,twin antibodies may better onescientists found...
1,1,Breakingviews - Corona Capital: Ford narrows t...,NEW YORK/HONG KONG/MILAN/LONDON (Reuters Break...,https://www.reuters.com/article/idUSKCN24V3NO,new corona capital daily column updated throug...
2,2,"Breakingviews - Corona Capital: Hockey, HBO, M...",NEW YORK/ZURICH/LONDON/MILAN/HONG KONG (Reuter...,https://www.reuters.com/article/idUSKBN23334I,new kong corona capital daily column updated t...
3,3,Sports-On this day... April 12,"(Reuters) - ON THIS DAY -- April 12April 12, 1...",https://www.reuters.com/article/idUSKCN21T08L,day april andre agassi boris becker pose respe...
4,4,Breakingviews - Corona Capital: Trouble in the...,NEW YORK/ZURICH/MUMBAI/LONDON (Reuters Breakin...,https://www.reuters.com/article/idUSKCN22A39X,new corona capital daily column updated throug...


### The power of LSH is that it can even scale sub-linearly using a Forest technique as the number of your items grow. As stated before, the goal is to find set(s) similar to a query set.

### The general method is to:

* Hash items such that similar items go into the same bucket with high probability.
* Restrict similarity search to the bucket associated with the query item.

In [2]:
!pip install datasketch

     |████████████████████████████████| 73 kB 157 kB/s eta 0:00:011
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

In [4]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [15]:
def get_forest(data, perms):
    start_time = time.time()  
    minhash = []   
    for text in data['cleaned_desc']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
    forest = MinHashLSHForest(num_perm=perms)   
    for i,m in enumerate(minhash):
        forest.add(i,m)       
    forest.index()   
    print('It took %s seconds to build forest.' %(time.time()-start_time))   
    return forest

In [16]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [17]:
#Number of Permutations
permutations = 100

#Number of Recommendations to return
num_recommendations = 5

forest = get_forest(ds, permutations)

It took 18.832992792129517 seconds to build forest.


In [18]:
num_recommendations = 10
title = ds['title'][11]
cleaned_desc = ds['cleaned_desc'][11]
result = predict(cleaned_desc, ds, permutations, num_recommendations+1, forest)
print("Article searched --> ",title)
for i in range(len(result)):
    print("Recomendation "+ str(i+1)+" --- " +result.iloc[i])
    print()
#print('\n Top Recommendation(s) is(are) \n', result[1:])

It took 0.011501789093017578 seconds to query forest.
Article searched -->  World Bank says no 'free ride' for commercial creditors on debt relief
Recomendation 1 --- India's electricity supply rises 7.1% in February

Recomendation 2 --- RPT-Asia Rice-More takers for cheaper India offers even as floods hit supply

Recomendation 3 --- Athletics: World championships moved to July 2022 to avoid Olympics clash

Recomendation 4 --- Nikkei slips, SoftBank Group tumbles after Nasdaq rout

Recomendation 5 --- UPDATE 1-World Bank's Malpass upbeat on prospects for progress on debt relief

Recomendation 6 --- RPT-Asia Rice-India prices rise as exporters grapple with floods, coronavirus

Recomendation 7 --- World Bank says no 'free ride' for commercial creditors on debt relief

Recomendation 8 --- Athletics-World Championships rescheduled to July 2022 over coronavirus

Recomendation 9 --- Golf: LPGA drops Kia Classic from 2020 schedule

Recomendation 10 --- Sport-Virus concern hits growing list of

LSH works better and faster as compaired to conventional tech that we used as it takes O(n) time to process as compaired to LSH which takes only O(log(n)) which is great. also how it works is that 
* first we creat no. of planes cutting the data points into respective compartements. ** we perform this using no.of Permutations 
* we then map them into a hash table using a minhash function 
* each new candidate is inserted into the hash table on by one and the table keeps on updating 
* inside the table for each key we find similarities using Jaccard Similarity.
* what we are duing is that when we hash them in the table and map them in the respective sections we then use HashLSH forest to find the top-k similarities in those respective sections. 
* The good part is that we can even find recomendations based on few words or the what ever text we enter.
* Drawback - if the partitions were not able to group the similar documents into same catogries then they may never be recomended. Also if the hashing is sometimes irregular that is some hash sections may be allocated to some not to any one else thus recommendation may not even be any where close to what is expected or asked for thus one needs to tune hypterparament no_of_permutations to optimize for all the docs and the type of corpous. 

Links -- 
* http://ekzhu.com/datasketch/lsh.html 
* http://ekzhu.com/datasketch/lshforest.html
* https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/ 
* https://www.youtube.com/watch?v=Arni-zkqMBA <-- very important
* https://www.youtube.com/watch?v=LqcwaW2YE_c  <-- very important

## What if user ask for news using some words.

In [19]:
num_recommendations = 5
title = "Facebook new companies"
result = predict(title, ds, permutations, num_recommendations, forest)
print("Article searched --> ",title)
for i in range(len(result)):
    print("Recomendation "+ str(i+1)+" --- " +result.iloc[i])
    print()

It took 0.007276296615600586 seconds to query forest.
Article searched -->  Facebook new companies
Recomendation 1 --- Brazil sports retailer Track & Field files for IPO

Recomendation 2 --- BRIEF-Facebook Says Won’t Be Attending This Year’s Mobile World Congress

Recomendation 3 --- What Matters: Russian trolls are back. Here's what you need to know

Recomendation 4 --- US considers blocking deals with China's largest chip maker

Recomendation 5 --- Salesforce beats quarterly revenue estimates on online demand



## Checking something abstract related to movies

In [20]:
num_recommendations = 5
title = "'Chariots of Fire,' 'Star Trek' actor Ben"
result = predict(title, ds, permutations, num_recommendations, forest)
print(title)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.007150411605834961 seconds to query forest.
'Chariots of Fire,' 'Star Trek' actor Ben

 Top Recommendation(s) is(are) 
 2883    Peru Congress votes against ouster of economy ...
1194    BRIEF-Nanjing Vishee Medical Technology Aims T...
1292                                Top Photos of the Day
819     BRIEF-Changchun High And New Technology Plans ...
1499                        Chadwick Boseman: 1976 - 2020
Name: title, dtype: object
